# IMPORT LIBRARIES AND LOAD DATA 

In [81]:
# import libraries
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor



In [82]:
# load data
df = pd.read_csv('../data/job_en_final.csv')

In [83]:
df.head()

,city_text,company_field,position,platform,exp_log_z,average_salary_log_z,skill_.net,skill_3d,skill_Not specified,skill_agile,...,skill_wpf,skill_xamarin,skill_xcode,skill_xml,group_skill_Data Systems,group_skill_Development Tools,group_skill_Languages,group_skill_Libs & Frameworks,group_skill_OS & Infrastructure,group_skill_Process & Methods
0,20,10,2,2,0.850560,2.903820,0,0,0,1,...,0,0,0,0,1,0,1,0,1,1
1,37,10,3,2,2.041137,2.903820,0,0,0,0,...,0,0,0,0,1,0,1,1,1,1
2,37,10,3,2,2.041137,2.903820,0,0,0,0,...,0,0,0,0,1,0,1,1,1,1
3,20,10,2,2,1.101143,2.903820,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,54,10,2,0,2.575805,2.877022,0,0,0,1,...,0,0,0,0,0,0,1,1,1,1


# LINEAR REGRESSION

In [84]:
X = df.drop(columns=['average_salary_log_z'])
y = df['average_salary_log_z']

In [85]:
from sklearn.model_selection import train_test_split

# Test 15%
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y,
    test_size=0.15,
    random_state=42
)

# Train / Validation
X_train, X_valid, y_train, y_valid = train_test_split(
    X_temp, y_temp,
    test_size=0.1765,  # ~15% tổng
    random_state=42
)


In [86]:
model = LinearRegression()
model.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [87]:
y_test_pred = model.predict(X_test)

print("TEST")
print("MAE:", mean_absolute_error(y_test, y_test_pred))
print("MSE:", mean_squared_error(y_test, y_test_pred))
print("R² :", r2_score(y_test, y_test_pred))


TEST
MAE: 0.5919954682784252
MSE: 0.6771955236182674
R² : 0.3312876718937052


# VIF

In [88]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np

X_vif = X_train.copy()

vif_df = pd.DataFrame()
vif_df['Feature'] = X_vif.columns
vif_df['VIF'] = [
    variance_inflation_factor(X_vif.values, i)
    for i in range(X_vif.shape[1])
]

vif_df.sort_values('VIF', ascending=False)


c:\Users\vthuy\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\regression\linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


,Feature,VIF
209,group_skill_Development Tools,11.274323
77,skill_html5,10.912997
41,skill_css3,10.776121
213,group_skill_Process & Methods,10.618009
2,position,10.541833
...,...,...
170,skill_sql-server,1.122340
188,skill_unit-testing,1.092765
200,skill_webgl,1.072365
146,skill_rtos,NaN


In [89]:
# delete if vif_value > 5
columns_to_delete = vif_df[vif_df['VIF'] > 5]['Feature'].tolist()
X_vif.drop(columns=columns_to_delete, inplace=True, errors='ignore')
X_train = X_vif.copy()
X_valid.drop(columns=columns_to_delete, inplace=True, errors='ignore')
X_test.drop(columns=columns_to_delete, inplace=True, errors='ignore')

# Lọc biến

In [90]:
corr = pd.concat([X_train, y_train], axis=1).corr()['average_salary_log_z']
corr = corr.drop('average_salary_log_z')

selected = corr[abs(corr) > 0.05].index


In [91]:
import statsmodels.api as sm

X_sm = sm.add_constant(X_train)
model = sm.OLS(y_train, X_sm).fit()

model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     average_salary_log_z   R-squared:                       0.490
Model:                              OLS   Adj. R-squared:                  0.438
Method:                   Least Squares   F-statistic:                     9.442
Date:                  Sun, 11 Jan 2026   Prob (F-statistic):          4.26e-172
Time:                          22:15:04   Log-Likelihood:                -2302.5
No. Observations:                  2116   AIC:                             4997.
Df Residuals:                      1920   BIC:                             6106.
Df Model:                           195                                         
Covariance Type:              nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.1333      0.047     -2.843      0.005      -0.225      -0.041
city_text               -0.0008      0.001     -0.856      0.392      -0.003       0.001
exp_log_z                0.5062      0.019     26.862      0.000       0.469       0.543
skill_.net               0.1246      0.184      0.676      0.499      -0.237       0.486
skill_3d                -0.0744      0.082     -0.912      0.362      -0.234       0.086
skill_Not specified      0.1265      0.062      2.048      0.041       0.005       0.248
skill_agile             -0.0560      0.072     -0.772      0.440      -0.198       0.086
skill_ajax               0.5692      0.423      1.346      0.178      -0.260       1.398
skill_algorithm          0.3255      0.160      2.033      0.042       0.012       0.639
skill_android           -0.0572      0.181     -0.316      0.752      -0.412       0.298
skill_angular           -0.1515      0.132     -1.148      0.251      -0.410       0.107
skill_angularjs          0.5623      0.365      1.542      0.123      -0.153       1.277
skill_ansible           -0.0326      0.299     -0.109      0.913      -0.620       0.555
skill_apache            -0.1764      0.234     -0.754      0.451      -0.635       0.282
skill_api               -0.0069      0.077     -0.089      0.929      -0.159       0.145
skill_appium            -0.8614      0.384     -2.243      0.025      -1.615      -0.108
skill_architecture       0.1826      0.073      2.496      0.013       0.039       0.326
skill_arm                1.2943      0.681      1.902      0.057      -0.040       2.629
skill_asp.net            0.0546      0.278      0.196      0.844      -0.490       0.599
skill_aws                0.1321      0.107      1.236      0.217      -0.077       0.342
skill_azure              0.0592      0.131      0.453      0.651      -0.197       0.316
skill_bash              -0.7195      0.228     -3.154      0.002      -1.167      -0.272
skill_bigdata            0.5990      0.569      1.054      0.292      -0.516       1.714
skill_blockchain         0.3630      0.189      1.923      0.055      -0.007       0.733
skill_boost              1.4304      0.390      3.666      0.000       0.665       2.196
skill_c                  0.1865      0.114      1.632      0.103      -0.038       0.411
skill_c#                -0.0779      0.175     -0.445      0.657      -0.421       0.266
skill_c++                0.7232      0.299      2.418      0.016       0.137       1.310
skill_cassandra          0.4664      0.395      1.179      0.238      -0.309       1.242
skill_chef               0.2499      0.516      0.484      0.628      -0.762       1.262
skill_cisco              0.4644      0.159      2.927      0.003       0.153       0.775
skill_citrix            -0.1907    

In [92]:
# delete if p_value > 0.05
p_values = model.pvalues
columns_to_delete = p_values[p_values > 0.05].index.tolist()
columns_to_delete = [col for col in columns_to_delete if col != 'const']
X_train.drop(columns=columns_to_delete, inplace=True, errors='ignore')
X_valid.drop(columns=columns_to_delete, inplace=True, errors='ignore')
X_test.drop(columns=columns_to_delete, inplace=True, errors='ignore')


In [93]:
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

(2116, 29)
(454, 29)
(454, 29)


# Linear – Ridge – Lasso

In [94]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd

models = {
    'Linear': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.01)
}

results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    results.append({
        'Model': name,
        'MAE': mean_absolute_error(y_test, y_pred),
        'MSE': mean_squared_error(y_test, y_pred),
        'R²': r2_score(y_test, y_pred)
    })

results_df = pd.DataFrame(results)
results_df


,Model,MAE,MSE,R²
0,Linear,0.596956,0.639334,0.368675
1,Ridge,0.592412,0.627056,0.380799
2,Lasso,0.597072,0.628377,0.379495


# MACHINE LEARNING MODELS

In [95]:
models = {
    "Linear": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.01),
    "Decision Tree": DecisionTreeRegressor(max_depth=6, random_state=42),
    "Random Forest": RandomForestRegressor(
        n_estimators=200, max_depth=8, random_state=42
    ),
    "Gradient Boosting": GradientBoostingRegressor(
        n_estimators=200, learning_rate=0.05, max_depth=4, random_state=42
    ),
    "SVR": SVR(kernel='rbf', C=10, epsilon=0.1)
}


In [96]:
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    results.append({
        "Model": name,
        "MAE": mean_absolute_error(y_test, y_pred),
        "MSE": mean_squared_error(y_test, y_pred),
        "R²": r2_score(y_test, y_pred)
    })

results_df = pd.DataFrame(results).sort_values("R²", ascending=False)
results_df


,Model,MAE,MSE,R²
6,SVR,0.571202,0.605598,0.401988
4,Random Forest,0.575235,0.610883,0.396770
5,Gradient Boosting,0.575593,0.612215,0.395454
1,Ridge,0.592412,0.627056,0.380799
2,Lasso,0.597072,0.628377,0.379495
0,Linear,0.596956,0.639334,0.368675
3,Decision Tree,0.608455,0.703944,0.304874


# FINE TUNE MODEL

In [97]:
param_grid = {
    "C": [0.1, 1, 10, 50],
    "epsilon": [0.01, 0.05, 0.1],
    "gamma": ['scale', 0.1, 0.01]
}


In [98]:
best_score = -1
best_model = None
results = []

for C in param_grid["C"]:
    for eps in param_grid["epsilon"]:
        for gamma in param_grid["gamma"]:
            model = SVR(kernel='rbf', C=C, epsilon=eps, gamma=gamma)
            model.fit(X_train, y_train)

            y_val_pred = model.predict(X_valid)
            r2 = r2_score(y_valid, y_val_pred)

            results.append({
                "C": C,
                "epsilon": eps,
                "gamma": gamma,
                "R2_valid": r2
            })

            if r2 > best_score:
                best_score = r2
                best_model = model


In [99]:
tune_results = pd.DataFrame(results).sort_values("R2_valid", ascending=False)
tune_results.head()


,C,epsilon,gamma,R2_valid
13,1.0,0.05,0.1,0.466370
10,1.0,0.01,0.1,0.466262
16,1.0,0.10,0.1,0.465571
15,1.0,0.10,scale,0.465170
12,1.0,0.05,scale,0.463225


In [100]:
y_test_pred = best_model.predict(X_test)

mae = mean_absolute_error(y_test, y_test_pred)
mse = mean_squared_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)

print("SVR (Best)")
print("MAE:", mae)
print("MSE:", mse)
print("R²:", r2)


SVR (Best)
MAE: 0.5489677087566819
MSE: 0.554209658762498
R²: 0.45273289877950385


In [101]:
print("Train R²:", r2_score(y_train, best_model.predict(X_train)))
print("Valid R²:", r2_score(y_valid, best_model.predict(X_valid)))
print("Test  R²:", r2_score(y_test, y_test_pred))


Train R²: 0.4807221333008316
Valid R²: 0.4663704862475383
Test  R²: 0.45273289877950385


# DEEP LEARNING

In [102]:
mlp = MLPRegressor(
    hidden_layer_sizes=(32, 16),
    activation='relu',
    solver='adam',
    alpha=0.0005,         
    learning_rate_init=0.001,
    max_iter=500,
    early_stopping=True,   
    validation_fraction=0.2,
    n_iter_no_change=20,
    random_state=42
)


In [103]:
mlp.fit(X_train, y_train)

y_test_pred = mlp.predict(X_test)

mae = mean_absolute_error(y_test, y_test_pred)
mse = mean_squared_error(y_test, y_test_pred)
r2  = r2_score(y_test, y_test_pred)

print("MLP Regression")
print("MAE:", mae)
print("MSE:", mse)
print("R²:", r2)


MLP Regression
MAE: 0.5647187688119408
MSE: 0.5785362637244353
R²: 0.4287110320192191


In [104]:
print("Train R²:", r2_score(y_train, mlp.predict(X_train)))
print("Valid R²:", r2_score(y_valid, mlp.predict(X_valid)))
print("Test  R²:", r2)


Train R²: 0.4569026923587026
Valid R²: 0.45347511335276913
Test  R²: 0.4287110320192191


# TUNE DEEP LEARNING

In [105]:
architectures = [
    (32,),
    (64,),
    (32, 16),
    (64, 32)
]

alphas = [0.0005, 0.001, 0.005, 0.01]
lrs = [0.001, 0.0005]


In [106]:
results = []
best_r2 = -1
best_model = None

for arch in architectures:
    for alpha in alphas:
        for lr in lrs:
            mlp = MLPRegressor(
                hidden_layer_sizes=arch,
                activation='relu',
                solver='adam',
                alpha=alpha,
                learning_rate_init=lr,
                max_iter=400,
                early_stopping=True,
                validation_fraction=0.2,
                n_iter_no_change=20,
                random_state=42
            )

            mlp.fit(X_train, y_train)
            r2_val = r2_score(y_valid, mlp.predict(X_valid))

            results.append({
                "Architecture": arch,
                "alpha": alpha,
                "lr": lr,
                "R2_valid": r2_val
            })

            if r2_val > best_r2:
                best_r2 = r2_val
                best_model = mlp


In [107]:
tune_df = pd.DataFrame(results).sort_values("R2_valid", ascending=False)
tune_df.head()


,Architecture,alpha,lr,R2_valid
2,"(32,)",0.0010,0.0010,0.456006
6,"(32,)",0.0100,0.0010,0.455663
3,"(32,)",0.0010,0.0005,0.455210
4,"(32,)",0.0050,0.0010,0.455070
1,"(32,)",0.0005,0.0005,0.454848


In [108]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_test_pred = best_model.predict(X_test)

print("Best Deep Learning Model")
print("MAE:", mean_absolute_error(y_test, y_test_pred))
print("MSE:", mean_squared_error(y_test, y_test_pred))
print("R² :", r2_score(y_test, y_test_pred))


Best Deep Learning Model
MAE: 0.5535680219710475
MSE: 0.5671873499865134
R² : 0.439917778464639


In [109]:
print("Train R²:", r2_score(y_train, best_model.predict(X_train)))
print("Valid R²:", r2_score(y_valid, best_model.predict(X_valid)))
print("Test  R²:", r2_score(y_test, y_test_pred))


Train R²: 0.4916012046921612
Valid R²: 0.45600559077881675
Test  R²: 0.439917778464639


# SAVE MODEL

In [112]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import joblib


In [116]:
X_train.head()

,exp_log_z,skill_Not specified,skill_algorithm,skill_appium,skill_architecture,skill_bash,skill_boost,skill_c++,skill_cisco,skill_cloud,...,skill_performance,skill_php,skill_saas,skill_scripting,skill_scrum,skill_security,skill_spark,skill_symfony,skill_unit-testing,skill_xcode
1426,-0.624102,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2760,-2.098765,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1986,-0.624102,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1547,0.850560,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
1455,-0.624102,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [119]:
import joblib
import pandas as pd
import os
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

num_features = X_train.shape[1]
print(f"📊 Dữ liệu X_train hiện tại có: {num_features} cột.")

print("🔄 Đang train lại SVR pipeline...")
model_pipeline = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
model_pipeline.fit(X_train, y_train)
print("✅ Train xong!")

if hasattr(X_train, 'columns'):
    real_columns = list(X_train.columns)
else:
    # Nếu X_train là numpy array (mất tên), ta phải lấy từ X gốc
    print("⚠️ X_train đang là array (mất tên cột). Lấy tên từ X gốc...")
    real_columns = list(X.columns)
    if len(real_columns) > num_features:
        print(f"⚠️ Cảnh báo: X gốc có {len(real_columns)} cột nhưng X_train chỉ có {num_features}.")
        print("👉 Đang giữ lại các cột khớp nhất...")
        real_columns = real_columns[:num_features]

print(f"📝 Danh sách cột sẽ lưu ({len(real_columns)} cột): {real_columns}")

save_dir = 'pipeline' 
if not os.path.exists(save_dir) and os.path.exists('../pipeline'):
    save_dir = '../pipeline'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Lưu Model
joblib.dump(model_pipeline, os.path.join(save_dir, 'model.pkl'))
# Lưu danh sách cột
joblib.dump(real_columns, os.path.join(save_dir, 'model_columns.pkl'))

print("------------------------------------------------------")
print(f"💾 ĐÃ LƯU THÀNH CÔNG VÀO: {save_dir}")
print(f"✅ Model và Danh sách cột đã đồng bộ: Cùng là {len(real_columns)} cột.")

📊 Dữ liệu X_train hiện tại có: 29 cột.
🔄 Đang train lại SVR pipeline...
✅ Train xong!
📝 Danh sách cột sẽ lưu (29 cột): ['exp_log_z', 'skill_Not specified', 'skill_algorithm', 'skill_appium', 'skill_architecture', 'skill_bash', 'skill_boost', 'skill_c++', 'skill_cisco', 'skill_cloud', 'skill_confluence', 'skill_design', 'skill_e-commerce', 'skill_firewall', 'skill_go', 'skill_hardware', 'skill_java', 'skill_jenkins', 'skill_linux', 'skill_performance', 'skill_php', 'skill_saas', 'skill_scripting', 'skill_scrum', 'skill_security', 'skill_spark', 'skill_symfony', 'skill_unit-testing', 'skill_xcode']
------------------------------------------------------
💾 ĐÃ LƯU THÀNH CÔNG VÀO: ../pipeline
✅ Model và Danh sách cột đã đồng bộ: Cùng là 29 cột.
